<a href="https://colab.research.google.com/github/yumehara/hackday_2022/blob/main/notebooks/resas_population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RESASから市区町村ごとの人口ピラミッドのデータを生成

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import requests
import pandas as pd

In [3]:
DIR_NAME = '/content/drive/MyDrive/hackday/'

In [5]:
def get_resas(url):
    resas_url = 'https://opendata.resas-portal.go.jp'
    API_KEY = 'XXXXXXX'
    response = requests.get(resas_url + url, headers={'X-API-Key': API_KEY})
    if response.status_code == requests.codes.ok:
        return response.json().get('result')
    else:
        raise Exception(f'resas error: {response.status_code} {response.text}')

In [9]:
# 都道府県コード
pref_code = get_resas('/api/v1/prefectures')
df_pref = pd.DataFrame(pref_code)

In [15]:
# 市区町村一覧
df_city = pd.DataFrame()
for index, row in df_pref.iterrows():
    city = get_resas(f'/api/v1/cities?prefCode={row["prefCode"]}')
    _tmp = pd.DataFrame(city)
    _tmp['prefName'] = row['prefName']
    df_city = pd.concat([df_city, _tmp])

print(df_city.shape)

(1922, 5)


In [17]:
df_city

,prefCode,cityCode,cityName,bigCityFlag,prefName
0,1,01100,札幌市,2,北海道
1,1,01101,札幌市中央区,1,北海道
2,1,01102,札幌市北区,1,北海道
3,1,01103,札幌市東区,1,北海道
4,1,01104,札幌市白石区,1,北海道
...,...,...,...,...,...
36,47,47361,久米島町,0,沖縄県
37,47,47362,八重瀬町,0,沖縄県
38,47,47375,多良間村,0,沖縄県
39,47,47381,竹富町,0,沖縄県


In [24]:
year = 2020
df_population = pd.DataFrame()
for index, row in df_city.iterrows():
    param = f'prefCode={row["prefCode"]}&cityCode={row["cityCode"]}&yearRight={year}&yearLeft={year}'
    pop_pyramid = get_resas(f'/api/v1/population/composition/pyramid?{param}')
    if pop_pyramid is not None:
        _tmp = pd.DataFrame(pop_pyramid['yearLeft']['data'])
        _tmp['prefCode'] = row['prefCode']
        _tmp['prefName'] = row['prefName']
        _tmp['cityCode'] = row['cityCode']
        _tmp['cityName'] = row['cityName']
        _tmp['year'] = year
        df_population = pd.concat([df_population, _tmp])
    else:
        print(row['prefCode'], row['cityCode'], row['cityName'])

print(df_population.shape)

1 01695 色丹村
1 01696 国後郡泊村
1 01697 留夜別村
1 01698 留別村
1 01699 紗那村
1 01700 蘂取村
7 07201 福島市
7 07202 会津若松市
7 07203 郡山市
7 07204 いわき市
7 07205 白河市
7 07207 須賀川市
7 07208 喜多方市
7 07209 相馬市
7 07210 二本松市
7 07211 田村市
7 07212 南相馬市
7 07213 伊達市
7 07214 本宮市
7 07301 桑折町
7 07303 国見町
7 07308 川俣町
7 07322 大玉村
7 07342 鏡石町
7 07344 天栄村
7 07362 下郷町
7 07364 檜枝岐村
7 07367 只見町
7 07368 南会津町
7 07402 北塩原村
7 07405 西会津町
7 07407 磐梯町
7 07408 猪苗代町
7 07421 会津坂下町
7 07422 湯川村
7 07423 柳津町
7 07444 三島町
7 07445 金山町
7 07446 昭和村
7 07447 会津美里町
7 07461 西郷村
7 07464 泉崎村
7 07465 中島村
7 07466 矢吹町
7 07481 棚倉町
7 07482 矢祭町
7 07483 塙町
7 07484 鮫川村
7 07501 石川町
7 07502 玉川村
7 07503 平田村
7 07504 浅川町
7 07505 古殿町
7 07521 三春町
7 07522 小野町
7 07541 広野町
7 07542 楢葉町
7 07543 富岡町
7 07544 川内村
7 07545 大熊町
7 07546 双葉町
7 07547 浪江町
7 07548 葛尾村
7 07561 新地町
7 07564 飯舘村
11 11101 さいたま市西区
11 11102 さいたま市北区
11 11103 さいたま市大宮区
11 11104 さいたま市見沼区
11 11105 さいたま市中央区
11 11106 さいたま市桜区
11 11107 さいたま市浦和区
11 11108 さいたま市南区
11 11109 さいたま市緑区
11 11110 さいたま市岩槻区
14 14151 相模原市緑区
14 14152 相模

In [26]:
df_population.to_csv(os.path.join(DIR_NAME, 'population_pyramid.csv'), index=False)